# 教程 1：构建和使用 CANN 模型

> **阅读时间**：约 25-30 分钟
> **难度**：初级
> **前提条件**：Python 基础、NumPy/JAX 数组操作

本教程将帮助您理解 CANNs 库中模型的构建方式以及如何使用内置的 CANN 模型。

---

## 目录

1. [BrainState框架介绍](#1-introduction-to-brainstate-framework)
2. [CANN1D实现分析](#2-cann1d-implementation-analysis)
3. [如何使用内置CANN模型](#3-how-to-use-built-in-cann-models)
4. [内置模型概览](#4-overview-of-built-in-models)
5. [后续步骤](#5-next-steps)

> **快速跳转**：如果你已经熟悉BrainState框架，请跳转到[如何使用内置CANN模型](#3-how-to-use-built-in-cann-models)。

---

## 1. BrainState 框架介绍

CANNs 库中的所有模型都是基于 [BrainState](https://brainstate.readthedocs.io/) 框架构建的。BrainState 是脑模拟生态系统中用于动力系统的核心框架，基于 JAX 构建，支持 JIT 编译和自动微分。

### 1.1 核心概念

在开始之前，你需要理解以下关键概念：

#### 动力学抽象

所有 CANN 模型都继承自 `brainstate.nn.Dynamics`，这是一个用于定义动力系统的基类。它提供了：
- 状态管理机制
- 时间步管理
- JIT 编译支持

In [1]:
import brainstate

class MyModel(brainstate.nn.Dynamics):
    def init_state(self):
        # 初始化状态变量
        pass

    def update(self, inp):
        # 定义单步动力学更新
        pass

#### 状态容器

BrainState 提供三种类型的状态容器来管理不同类型的变量：

| 容器类型 | 用途 | 示例 |
|---------------|---------|---------|
| `brainstate.State` | 外部输入或可观测状态 | 外部刺激 `inp` |
| `brainstate.HiddenState` | 内部隐藏状态 | 膜电位 `u`、放电率 `r` |
| `brainstate.ParamState` | 可学习参数 | 突触权重 `W` |

In [2]:
def init_state(self):
    # 隐藏状态：神经元膜电位
    self.u = brainstate.HiddenState(u.math.zeros(self.num))
    # 隐藏状态：神经元放电率
    self.r = brainstate.HiddenState(u.math.zeros(self.num))
    # 外部输入状态
    self.inp = brainstate.State(u.math.zeros(self.num))

#### 时间步管理

BrainState 通过 `brainstate.environ` 统一管理模拟时间步：

In [3]:
import brainstate

# 设置模拟时间步长（单位：毫秒）
brainstate.environ.set(dt=0.1)

# 获取模型中的当前时间步长
dt = brainstate.environ.get_dt()

> **重要**: 在运行任何模拟之前，必须设置时间步长 `dt`，否则会出现错误。

#### 进一步学习

要了解更多关于 BrainState 框架的信息，请参阅：
- [BrainState 官方文档](https://brainstate.readthedocs.io/)
- [循环和条件教程](https://brainstate.readthedocs.io/tutorials/transforms/05_loops_conditions.html)

---

## 2. CANN1D 实现分析

让我们以 `CANN1D` 为例来理解完整的 CANN 模型是如何实现的。

### 2.1 模型继承结构

```
brainstate.nn.Dynamics
    └── BasicModel
        └── BaseCANN
            └── BaseCANN1D
                └── CANN1D
```

### 2.2 初始化方法 `__init__`

`CANN1D` 初始化方法定义了所有模型参数：

In [ ]:
class CANN1D(BaseCANN1D):
    def __init__(
        self,
        num: int,           # 神经元数量
        tau: float = 1.0,   # 时间常数
        k: float = 8.1,     # 全局抑制强度
        a: float = 0.5,     # 连接宽度
        A: float = 10,      # 外部输入幅度
        J0: float = 4.0,    # 突触连接强度
        z_min: float = -π,  # 特征空间最小值
        z_max: float = π,   # 特征空间最大值
        **kwargs,
    ):
        ...

这些参数控制网络的动力学行为。我们将在[教程 4：参数效应](./04_parameter_effects.ipynb)中详细探索每个参数的效果。

### 2.3 连接矩阵生成 `make_conn`

`make_conn` 方法生成神经元之间的连接矩阵。CANN 使用高斯连接核，使得具有相似特征偏好的神经元具有更强的兴奋性连接：

In [ ]:
def make_conn(self):
    # 计算所有神经元对之间的距离
    x_left = u.math.reshape(self.x, (-1, 1))
    x_right = u.math.repeat(self.x.reshape((1, -1)), len(self.x), axis=0)
    d = self.dist(x_left - x_right)

    # 使用高斯函数计算连接强度
    return (
        self.J0
        * u.math.exp(-0.5 * u.math.square(d / self.a))
        / (u.math.sqrt(2 * u.math.pi) * self.a)
    )

### 2.4 刺激生成 `get_stimulus_by_pos`

`get_stimulus_by_pos` 基于特征空间中的给定位置生成外部刺激（高斯形状的凸起）：

In [ ]:
def get_stimulus_by_pos(self, pos):
    return self.A * u.math.exp(
        -0.25 * u.math.square(self.dist(self.x - pos) / self.a)
    )

此方法由任务模块调用以生成输入数据。

### 2.5 状态初始化 `init_state`

`init_state` 方法初始化模型的所有状态变量：

In [ ]:
def init_state(self, *args, **kwargs):
    # 放电率
    self.r = brainstate.HiddenState(u.math.zeros(self.varshape))
    # 膜电位（突触输入）
    self.u = brainstate.HiddenState(u.math.zeros(self.varshape))
    # 外部输入
    self.inp = brainstate.State(u.math.zeros(self.varshape))

> **重要提示**：在运行模拟之前，您必须调用 `model.init_state()` 来初始化状态，否则会出现错误。

### 2.6 动力学更新 `update`

`update` 方法定义了网络的单步动力学更新：

In [ ]:
def update(self, inp):
    self.inp.value = inp

    # 计算放电率（除法归一化）
    r1 = u.math.square(self.u.value)
    r2 = 1.0 + self.k * u.math.sum(r1)
    self.r.value = r1 / r2

    # 计算递归输入
    Irec = u.math.dot(self.conn_mat, self.r.value)

    # 使用欧拉方法更新膜电位
    self.u.value += (
        (-self.u.value + Irec + self.inp.value)
        / self.tau * brainstate.environ.get_dt()
    )

---

## 3. 如何使用内置 CANN 模型

现在让我们学习如何实际使用内置 CANN 模型。

### 3.1 基本使用工作流

In [17]:
import brainstate
import brainunit as u
from canns.models.basic import CANN1D

# 步骤 1: 设置时间步
brainstate.environ.set(dt=0.1)

# 步骤 2: 创建模型实例
model = CANN1D(
    num=256,      # 256 个神经元
    tau=1.0,      # 时间常数
    k=8.1,        # 全局抑制
    a=0.5,        # 连接宽度
    A=10,         # 输入振幅
    J0=4.0,       # 连接强度
)

# 步骤 3: 初始化状态
model.init_state()

# 步骤 4: 查看模型信息
print(f"神经元数量: {model.shape}")
print(f"特征空间范围: [{model.z_min}, {model.z_max}]")
print(f"连接矩阵形状: {model.conn_mat.shape}")

Number of neurons: (256,)
Feature space range: [-3.141592653589793, 3.141592653589793]
Connection matrix shape: (256, 256)


### 3.2 运行单步更新

In [18]:
# 在pos=0处生成外部刺激
pos = 0.0
stimulus = model.get_stimulus_by_pos(pos)

# 运行两步更新
model(stimulus)	# 或者你可以显式调用 model.update(stimulus)
model(stimulus)

# 查看当前状态
print(f"放电率形状: {model.r.value.shape}")
print(f"最大放电率: {u.math.max(model.r.value):.4f}")
print(f"最大膜电位: {u.math.max(model.u.value):.4f}")

Firing rate shape: (256,)
Max firing rate: 0.0024
Max membrane potential: 1.9275


### 3.3 完整示例

以下是创建和测试 CANN1D 模型的完整示例：

In [21]:
import brainstate
import brainunit as u
from canns.models.basic import CANN1D

# 设置环境
brainstate.environ.set(dt=0.1)

# 创建模型
model = CANN1D(num=256, tau=1.0, k=8.1, a=0.5, A=10, J0=4.0)
model.init_state()

# 打印基本模型信息
print("=" * 50)
print("CANN1D 模型信息")
print("=" * 50)
print(f"神经元数量: {model.shape}")
print(f"时间常数 tau: {model.tau}")
print(f"全局抑制 k: {model.k}")
print(f"连接宽度 a: {model.a}")
print(f"输入幅度 A: {model.A}")
print(f"连接强度 J0: {model.J0}")
print(f"特征空间: [{model.z_min:.2f}, {model.z_max:.2f}]")
print(f"神经密度 rho: {model.rho:.2f}")

# 测试刺激生成
pos = 0.5
stimulus = model.get_stimulus_by_pos(pos)
print(f"\n刺激位置: {pos}")
print(f"刺激形状: {stimulus.shape}")
print(f"最大刺激值: {u.math.max(stimulus):.4f}")

# 运行多个更新步骤
print("\n运行 100 个更新步骤...")
for _ in range(100):
    model(stimulus)

print(f"最大放电率: {u.math.max(model.r.value):.6f}")
print(f"最大膜电位: {u.math.max(model.u.value):.6f}")

CANN1D Model Information
Number of neurons: (256,)
Time constant tau: 1.0
Global inhibition k: 8.1
Connection width a: 0.5
Input amplitude A: 10
Connection strength J0: 4.0
Feature space: [-3.14, 3.14]
Neural density rho: 40.74

Stimulus position: 0.5
Stimulus shape: (256,)
Max stimulus value: 9.9997

Running 100 update steps...
Max firing rate: 0.002427
Max membrane potential: 10.278063


---

## 4. 内置模型概览

CANNs 库提供三类内置模型：

### 基础模型

标准 CANN 实现及其变体：
- `CANN1D` - 1D 连续吸引子神经网络
- `CANN1D_SFA` - 带有尖峰频率自适应的 CANN1D
- `CANN2D` - 2D 连续吸引子神经网络
- `CANN2D_SFA` - 带有 SFA 的 CANN2D
- 分层路径整合网络（网格细胞、位置细胞、带状细胞等）
- theta 扫描模型

### 脑启发模型

基于神经科学原理的学习模型：
- Hopfield 网络
- ...

### 混合模型

CANN 与人工神经网络的组合（开发中）。

> **详细信息**：查看 [第 3 层核心概念 - 模型集合](../../docs/en/2_core_concepts/02_model_collections.rst) 了解完整的模型列表和使用案例。

---

## 5. 后续步骤

恭喜您完成了第一个教程！您现在理解了：
- BrainState 框架的核心概念
- CANN1D 的实现结构
- 如何创建和初始化内置模型

### 继续学习

- **下一个教程**: [教程 2：任务生成和 CANN 模拟](./02_task_and_simulation.ipynb) - 学习如何生成任务数据和运行完整模拟
- **了解更多关于 BrainState 的信息**: 访问 [BrainState ReadTheDocs](https://brainstate.readthedocs.io/)
- **查看所有可用模型**: 检查 [模型集合](../../docs/en/2_core_concepts/02_model_collections.rst)